## Part V: Training a Model with Data Programming

In [1]:
# If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ features snorkel.db');

In [2]:
%load_ext autoreload
%autoreload 2

from snorkel import SnorkelSession
session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from snorkel.models import candidate_subclass

Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

### Loading the `CandidateSet`, feature matrix, and label matrix

In [4]:
from snorkel.models import CandidateSet
train = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()

In [5]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()

%time F_train = feature_manager.load(session, train, 'Train Features')

CPU times: user 2min 40s, sys: 13.1 s, total: 2min 53s
Wall time: 3min


In [6]:
from snorkel.annotations import LabelManager

label_manager = LabelManager()

%time L_train = label_manager.load(session, train, 'LF Labels')

CPU times: user 3.19 s, sys: 722 ms, total: 3.91 s
Wall time: 4.09 s


## Train Generative Model

In [7]:
from snorkel.learning import NaiveBayes

gen_model = NaiveBayes()
# gen_model.train(L_train, n_iter=3000, rate=1e-5)
gen_model.train(L_train, n_iter=15000, rate=1e-2)

Training marginals (!= 0.5):	75908
Features:			18
Begin training for rate=0.01, mu=1e-06
	Learning epoch = 0	Gradient mag. = 0.368645
	Learning epoch = 250	Gradient mag. = 0.491158
	Learning epoch = 500	Gradient mag. = 0.618887
	Learning epoch = 750	Gradient mag. = 0.751792
	Learning epoch = 1000	Gradient mag. = 0.831461
	Learning epoch = 1250	Gradient mag. = 0.741360
	Learning epoch = 1500	Gradient mag. = 0.626068
	Learning epoch = 1750	Gradient mag. = 0.538426
	Learning epoch = 2000	Gradient mag. = 0.473630
	Learning epoch = 2250	Gradient mag. = 0.422921
	Learning epoch = 2500	Gradient mag. = 0.381266
	Learning epoch = 2750	Gradient mag. = 0.346783
	Learning epoch = 3000	Gradient mag. = 0.319224
	Learning epoch = 3250	Gradient mag. = 0.298483
	Learning epoch = 3500	Gradient mag. = 0.283973
	Learning epoch = 3750	Gradient mag. = 0.265775
	Learning epoch = 4000	Gradient mag. = 0.244649
	Learning epoch = 4250	Gradient mag. = 0.226658
	Learning epoch = 4500	Gradient mag. = 0.211249
	Lear

In [8]:
gen_model.save(session, 'Generative Params')

In [9]:
train_marginals = gen_model.marginals(L_train)

## Train Discriminative Model

In [10]:
from snorkel.learning import LogReg

disc_model = LogReg()
disc_model.train(F_train, train_marginals, n_iter=4500, rate=1e-3)

Training marginals (!= 0.5):	59699
Features:			19168
Using gradient descent...
	Learning epoch = 0	Step size = 0.001
	Loss = 41380.193532	Gradient magnitude = 70993.801698
	Learning epoch = 100	Step size = 0.000904792147114
	Loss = 39612.814287	Gradient magnitude = 5381.482894
	Learning epoch = 200	Step size = 0.000818648829479
	Loss = 212963.945946	Gradient magnitude = 15710.857944
	Learning epoch = 300	Step size = 0.000740707032156
	Loss = 77940.491218	Gradient magnitude = 2496.459661
	Learning epoch = 400	Step size = 0.000670185906007
	Loss = 25706.122873	Gradient magnitude = 2301.106650
	Learning epoch = 500	Step size = 0.000606378944861
	Loss = 29477.333460	Gradient magnitude = 2304.353107
	Learning epoch = 600	Step size = 0.000548646907485
	Loss = 51481.179098	Gradient magnitude = 2369.000257
	Learning epoch = 700	Step size = 0.000496411413431
	Loss = 16233.732067	Gradient magnitude = 2134.240654
	Learning epoch = 800	Step size = 0.00044914914861
	Loss = 14999.072146	Gradient mag

In [11]:
disc_model.w.shape

(19168,)

In [12]:
%time disc_model.save(session, "Discriminative Params")

CPU times: user 3.96 s, sys: 98.3 ms, total: 4.05 s
Wall time: 4.08 s


## Assess Performance on Development Set

In [13]:
from snorkel.models import CandidateSet
dev = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Development Candidates').one()

### Loading the Development Feature Matrix

In [14]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()
%time F_dev = feature_manager.load(session, dev, 'Train Features')

CPU times: user 1min 34s, sys: 7.06 s, total: 1min 41s
Wall time: 1min 43s


In [15]:
train_gold = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates -- Gold').one()

In [16]:
# from snorkel.loaders import create_or_fetch
# from snorkel.models import Label, AnnotationKeySet
# annotation_key_name = "Hardware Development Labels - Gold"
# annotation_key = create_or_fetch(session, AnnotationKey, annotation_key_name)
# session.query(Label).filter(Label.key == annotation_key).count()

In [17]:
from snorkel.models import CandidateSet
from snorkel.annotations import LabelManager
label_manager = LabelManager()
# from snorkel.utils import get_ORM_instance
# dev_labels = get_ORM_instance(AnnotationKeySet, session, 'Hardware Development Labels - Gold')
L_dev = label_manager.load(session, dev, 'Hardware Development Labels -- Gold')

In [18]:
gold_dev_set = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Development Candidates -- Gold').one()

In [19]:
F_dev.shape

(18971, 19168)

In [20]:
tp, fp, tn, fn = disc_model.score(F_dev, L_dev, gold_dev_set)

Calibration plot:
Recall-corrected Noise-aware Model
Pos. class accuracy: 0.654894046418
Neg. class accuracy: 0.719236287385
Corpus Precision 0.718
Corpus Recall    0.655
Corpus F1        0.685
----------------------------------------
TP: 6490 | FP: 2544 | TN: 6517 | FN: 3420

Recall-corrected Noise-aware Model
Pos. class accuracy: 0.654894046418
Neg. class accuracy: 0.719236287385
Corpus Precision 0.718
Corpus Recall    0.655
Corpus F1        0.685
----------------------------------------
TP: 6490 | FP: 2544 | TN: 6517 | FN: 3420



/Users/bradenhancock/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


In [21]:
from hardware_utils import part_error_analysis, table_info
#     print c[0]
#     print c[0].parent
#     print (c[0].parent.row_num, c[0].parent.col_num)
#     print c[1]
#     print c[1].parent
#     print (c[1].parent.row_num, c[1].parent.col_num)

In [22]:
# for c in list(fp)[:5]:
#     print c
# c = list(fp)[10]
# error_analysis(c)
# c = list(fp)[10]
# part_error_analysis(c)
# print c.temp.parent.row_num

# for c in list(fp)[:5]:
#     print c
# print c.temp.parent.html_anc_tags

In [23]:
from snorkel.models import Corpus
from hardware_utils import entity_level_f1
import os

gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
corpus = session.query(Corpus).filter(Corpus.name == 'Hardware Development').one()
(TP, FP, FN) = entity_level_f1(tp, fp, tn, fn, gold_file, corpus, 'stg_temp_min')

Scoring on Entity-Level Gold Data
Corpus Precision 0.822
Corpus Recall    1.0
Corpus F1        0.902
----------------------------------------
TP: 152 | FP: 33 | FN: 0



In [24]:
print FN

set([])


In [25]:
from pprint import pprint
pprint(FP)

set([(u'ONSMS04099-1', u'BC237', u'-65'),
     (u'ONSMS04099-1', u'BC237A', u'-65'),
     (u'ONSMS04099-1', u'BC237B', u'-65'),
     (u'ONSMS04099-1', u'BC237C', u'-65'),
     (u'ONSMS04099-1', u'BC238', u'-65'),
     (u'ONSMS04099-1', u'BC238B', u'-65'),
     (u'ONSMS04099-1', u'BC238C', u'-65'),
     (u'ONSMS04099-1', u'BC239', u'-65'),
     (u'ONSMS04099-1', u'BC239C', u'-65'),
     (u'PHGLS19500-1', u'MMBT3904', u'-55'),
     (u'PNJIS01593-1', u'BC856', u'-65'),
     (u'PNJIS01593-1', u'BC856A', u'-65'),
     (u'PNJIS01593-1', u'BC856B', u'-65'),
     (u'PNJIS01593-1', u'BC857', u'-65'),
     (u'PNJIS01593-1', u'BC857A', u'-65'),
     (u'PNJIS01593-1', u'BC857B', u'-65'),
     (u'PNJIS01593-1', u'BC857C', u'-65'),
     (u'PNJIS01593-1', u'BC858', u'-65'),
     (u'PNJIS01593-1', u'BC858A', u'-65'),
     (u'PNJIS01593-1', u'BC858B', u'-65'),
     (u'PNJIS01593-1', u'BC858C', u'-65'),
     (u'PNJIS01593-1', u'BC859', u'-65'),
     (u'PNJIS01593-1', u'BC859B', u'-65'),
     (u'PNJIS015

In [26]:
import os
os.system('cp snorkel.db snorkel.db\ final');

In [27]:
from pprint import pprint
all_weights = disc_model.get_feature_weights(F_train)

In [28]:
pprint(all_weights[:60])

[(AnnotationKey (TAB_e2_ROW_LEMMAS_[storage]), 27.779176006753143),
 (AnnotationKey (TAB_e2_ROW_LEMMAS_[storage temperature]), 23.197103524491123),
 (AnnotationKey (TAB_e2_ROW_LEMMAS_[stg]), 20.535407429044525),
 (AnnotationKey (TAB_e2_ROW_LEMMAS_[t stg]), 20.53249686118379),
 (AnnotationKey (TAB_e2_ROW_LEMMAS_[temperature]), 18.980118208578798),
 (AnnotationKey (TAB_e2_ROW_LEMMAS_[t]), 18.173587020149323),
 (AnnotationKey (TAB_e2_COL_LEMMAS_[80]), 10.401075548903389),
 (AnnotationKey (TAB_e2_COL_LEMMAS_[100]), 10.368437239665312),
 (AnnotationKey (TAB_e2_ROW_LEMMAS_[range]), 10.30837547022631),
 (AnnotationKey (TAB_e2_ROW_LEMMAS_[temperature range]), 10.30837547022631),
 (AnnotationKey (TAB_e2_COL_LEMMAS_[150]), 10.043978795889958),
 (AnnotationKey (TAB_e2_COL_LEMMAS_[65]), 8.884559575925453),
 (AnnotationKey (TAB_e2_COL_NUM_[2]), 8.828013609235567),
 (AnnotationKey (TAB_e2_W_LEFT_POS_3_[CD TO CD]), 8.62074540597933),
 (AnnotationKey (TAB_e2_W_LEFT_POS_2_[TO CD]), 8.62074540597933),
 

In [29]:
fp_zetx = set()
print(len(fn))
for f in fn:
    if (str(f.part.parent.document) == "Document PNJIS00254-1"):
        fp_zetx.add(f)

print(len(fp_zetx))

3420
0


In [30]:
sample = fp_zetx.pop()
print sample
print sample.temp.parent

KeyError: 'pop from an empty set'

In [ ]:
from snorkel.utils import get_keys_by_candidate
for a in sorted(get_keys_by_candidate(F_dev, sample)):
    print a

In [ ]:
from snorkel.lf_helpers import *
def LF_min_col(c):
    return -1 if 'min' in get_col_ngrams(c.temp) or 'min.' in get_col_ngrams(c.temp) else 0

LF_min_col(sample)

The End.